In [ ]:
import sys
import math
import os
import pandas as pd
import time
import cv2
from openpyxl import load_workbook
import numpy as np
import mysql.connector as mc
from scipy.stats import skew, kurtosis
import getopt
from skimage.feature import greycomatrix, greycoprops

In [ ]:
# path of the folder containing the images
sourcePath = '05-transparent/20230117/ARABIKA_nobg'

In [ ]:
def ftr_extract(sourcePath):
    for imagePath in os.listdir(sourcePath):

        # imagePath contains name of the image
        inputPath = os.path.join(sourcePath, imagePath)

        basename = os.path.basename(inputPath)

        image = cv2.imread(inputPath)
        h,w,c = image.shape

        # --- COLOR FEATURES

        b, g, r = cv2.split(image)
        N = r.shape[0] * r.shape[1]
        # Mean
        r_mean = np.mean(r); g_mean = np.mean(g); b_mean = np.mean(b)
        r_sd = np.std(r); g_sd = np.std(g); b_sd = np.std(b)
        r_var = np.var(r); g_var = np.var(g); b_var = np.var(b)
        r_skew = skew(r.reshape(-1)); g_skew = skew(g.reshape(-1)); b_skew = skew(b.reshape(-1))
        r_kurt = kurtosis(r.reshape(-1)); g_kurt = kurtosis(g.reshape(-1)); b_kurt = kurtosis(b.reshape(-1))

        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        h_mean = np.mean(h); s_mean = np.mean(s); v_mean = np.mean(v)
        h_sd = np.std(h); s_sd = np.std(s); v_sd = np.std(v)
        h_var = np.var(h); s_var = np.var(s); v_var = np.var(v)
        h_skew = skew(h.reshape(-1)); s_skew = skew(s.reshape(-1)); v_skew = skew(v.reshape(-1))
        h_kurt = kurtosis(h.reshape(-1)); s_kurt = kurtosis(s.reshape(-1)); v_kurt = kurtosis(v.reshape(-1))

        # Convert the BGR image to Lab
        Lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
        L, a, b = cv2.split(Lab)
        CIEL_mean = np.mean(L); CIEa_mean = np.mean(a); CIEb_mean = np.mean(b)
        CIEL_sd = np.std(L); CIEa_sd = np.std(a); CIEb_sd = np.std(b)
        CIEL_var = np.var(L); CIEa_var = np.var(a); CIEb_var = np.var(b)
        CIEL_skew = skew(L.reshape(-1)); CIEa_skew = skew(a.reshape(-1)); CIEb_skew = skew(b.reshape(-1))
        CIEL_kurt = kurtosis(L.reshape(-1));  CIEa_kurt = kurtosis(a.reshape(-1)); CIEb_kurt = kurtosis(b.reshape(-1))

        # --- TEXTURAL FEATURES

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        angle1 = 0; angle2 = np.pi/4; angle3 = np.pi/2; angle4 = 3 * np.pi / 4;

        graycom1 = greycomatrix(gray, [1], [angle1], levels=256)
        contrast0 = greycoprops(graycom1, 'contrast')
        dissimilarity0 = greycoprops(graycom1, 'dissimilarity')
        homogeneity0 = greycoprops(graycom1, 'homogeneity')
        asm0 = greycoprops(graycom1, 'ASM')
        energy0 = greycoprops(graycom1, 'energy')
        correlation0 = greycoprops(graycom1, 'correlation')

        graycom2 = greycomatrix(gray, [1], [angle2], levels=256)
        contrast45 = greycoprops(graycom2, 'contrast')
        dissimilarity45 = greycoprops(graycom2, 'dissimilarity')
        homogeneity45 = greycoprops(graycom2, 'homogeneity')
        asm45 = greycoprops(graycom2, 'ASM')
        energy45 = greycoprops(graycom2, 'energy')
        correlation45 = greycoprops(graycom2, 'correlation')

        graycom3 = greycomatrix(gray, [1], [angle3], levels=256)
        contrast90 = greycoprops(graycom3, 'contrast')
        dissimilarity90 = greycoprops(graycom3, 'dissimilarity')
        homogeneity90 = greycoprops(graycom3, 'homogeneity')
        asm90 = greycoprops(graycom3, 'ASM')
        energy90 = greycoprops(graycom3, 'energy')
        correlation90 = greycoprops(graycom3, 'correlation')

        graycom4 = greycomatrix(gray, [1], [angle4], levels=256)
        contrast135 = greycoprops(graycom4, 'contrast')
        dissimilarity135 = greycoprops(graycom4, 'dissimilarity')
        homogeneity135 = greycoprops(graycom4, 'homogeneity')
        asm135 = greycoprops(graycom4, 'ASM')
        energy135 = greycoprops(graycom4, 'energy')
        correlation135 = greycoprops(graycom4, 'correlation')

        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, c = rgb.shape
        rectangle = (0, 0, h - 1, w - 1)
        mask = np.zeros(image.shape[:2], np.uint8)
        bgdModel = np.zeros((1, 65), np.float64)
        fgdModel = np.zeros((1, 65), np.float64)
        cv2.grabCut(image, mask, rectangle, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
        mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
        image_ = image * mask2[:, :, np.newaxis]

        # Make Transparent
        tmp_ = cv2.cvtColor(image_, cv2.COLOR_BGR2GRAY)
        th, a_ = cv2.threshold(tmp_, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        b, g, r, = cv2.split(image_)
        rgba = [r, g, b, a_]
        nobg = cv2.merge(rgba, 4)
        image = cv2.cvtColor(nobg, cv2.COLOR_RGB2BGR)

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blank = np.zeros(image.shape, dtype='uint8')
        ret, thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)
        # edges = cv2.Canny(thresh, 50, 200)

        contour, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cv2.drawContours(blank, contour, -1, (0, 0, 255), 1)
        # numContour = len(contour)

        cnt = contour[0]

        # PERIMETER
        perimeter = cv2.arcLength(cnt, True)

        # AREA
        area = cv2.contourArea(cnt, False)

        # ROUNDNESS
        roundness = 4 * math.pi * area / (perimeter * perimeter)

        # EQUIVALENT DIAMETER --- the diameter of the circle whose area is same as the contour area.
        equiv_diameter = np.sqrt(4 * area / np.pi)

        # COMPLEXITY
        complexity = (perimeter * perimeter) / area

        # ASPECT RATIO  --- the ratio of width to height of bounding rect of the object
        rx, ry, width, height = cv2.boundingRect(cnt)
        if height > width:
            objWidth = width
            objHeight = height
        else:
            objWidth = height
            objHeight = width
        aspect_ratio = objWidth / objHeight

        # EXTENT --- the ratio of contour area to bounding rectangle area
        rect_area = width * height
        extent = area / rect_area

        # CONVEX HULL
        convex = cv2.convexHull(cnt)
        convex_perimeter = cv2.arcLength(convex, True)
        ConvexArea_ = cv2.contourArea(convex, False)

        # SOLIDITY --- the ratio of contour area to its convex hull area.
        solidity = area / ConvexArea_

        (px, py), (Ma, ma), angle = cv2.fitEllipse(cnt)
        axLengthRatio = Ma / ma
        MinorAxis = ma
        MajorAxis = Ma

        compactness = math.sqrt(equiv_diameter / Ma)

        SF1 = Ma / area

        SF2 = ma / area

        SF3 = area / ((Ma / 2) * (Ma / 2) * np.pi)

        SF4 = area / ((Ma / 2) * (ma / 2) * np.pi)
        label = 'ARABICA'

        mydb = mc.connect(
            host="localhost",
            user="root",
            password="",
            database="cvs"
        )
        mycursor = mydb.cursor()

        sql = "INSERT INTO features(r_mean, g_mean, b_mean, r_sd, g_sd, b_sd, r_var, g_var, b_var, r_skew, g_skew, b_skew, r_kurt, g_kurt, b_kurt," \
              "h_mean, s_mean, v_mean, h_sd, s_sd, v_sd, h_var, s_var, v_var, h_skew, s_skew, v_skew, h_kurt, s_kurt, v_kurt," \
              "CIEL_mean, CIEa_mean, CIEb_mean, CIEL_sd, CIEa_sd, CIEb_sd, CIEL_var, CIEa_var, CIEb_var, CIEL_skew, CIEa_skew, CIEb_skew, CIEL_kurt, CIEa_kurt, CIEb_kurt," \
              "contrast0, contrast45, contrast90, contrast135, dissimilarity0, dissimilarity45, dissimilarity90, dissimilarity135," \
              "homogeneity0, homogeneity45, homogeneity90, homogeneity135, asm0, asm45, asm90, asm135, energy0, energy45, energy90, energy135," \
              "correlation0, correlation45, correlation90, correlation135, " \
              "area, perimeter, roundness, equiv_diameter, complexity, aspect_ratio, extent, solidity," \
              "minor_axis_length, major_axis_length, compactness, convex_area, SF1, SF2, SF3, SF4) VALUES( " \
              " %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, " \
              " %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, " \
              " %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, " \
              " %s, %s, %s, %s, %s, %s, %s, %s, " \
              " %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, " \
              " %s, %s, %s, %s, " \
              " %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val = [float(r_mean), float(g_mean), float(b_mean), float(r_sd), float(g_sd), float(b_sd), float(r_var), float(g_var), float(b_var), float(r_skew), float(g_skew), float(b_skew), float(r_kurt), float(g_kurt), float(b_kurt),
                float(h_mean), float(s_mean), float(v_mean), float(h_sd), float(s_sd), float(v_sd), float(h_var), float(s_var), float(v_var), float(h_skew), float(s_skew), float(v_skew), float(h_kurt), float(s_kurt), float(v_kurt),
                float(CIEL_mean), float(CIEa_mean), float(CIEb_mean), float(CIEL_sd), float(CIEa_sd), float(CIEb_sd), float(CIEL_var), float(CIEa_var), float(CIEb_var), float(CIEL_skew), float(CIEa_skew), float(CIEb_skew), float(CIEL_kurt), float(CIEa_kurt), float(CIEb_kurt),
                float(contrast0), float(contrast45), float(contrast90), float(contrast135), float(dissimilarity0), float(dissimilarity45), float(dissimilarity90), float(dissimilarity135),
                float(homogeneity0), float(homogeneity45), float(homogeneity90), float(homogeneity135), float(asm0), float(asm45), float(asm90), float(asm135), float(energy0), float(energy45), float(energy90), float(energy135),
                float(correlation0), float(correlation45), float(correlation90), float(correlation135),
                float(area), float(perimeter), float(roundness), float(equiv_diameter), float(complexity), float(aspect_ratio), float(extent),
                float(solidity),  float(MinorAxis), float(MajorAxis), float(compactness), float(ConvexArea_), float(SF1), float(SF2), float(SF3), float(SF4)]

        mycursor.execute(sql, val)
        mydb.commit()


        #df = pd.DataFrame(list(zip([basename], [r_mean])))

        #print(df)
        #df.to_excel('database.xlsx')


In [ ]:
if __name__ == '__main__':
    ftr_extract(sourcePath)